In [14]:
import yfinance as yf

import pandas as pd

# Load your SP500.csv
csv_path = "/Users/raghav/Downloads/sp500tickers/SP500.csv"  # update if needed
df = pd.read_csv(csv_path)

# Get tickers as list
tickers = df["Symbol"].dropna().unique().tolist()

print("Total tickers:", len(tickers))
print("Sample:", tickers[:10])  # just to preview



Total tickers: 505
Sample: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ATVI', 'AYI', 'ADBE', 'AAP', 'AMD']


In [18]:
import yfinance as yf
import pandas as pd

# Download data for all tickers
data = yf.download(tickers, period="1d")["Close"]

# Get the last available row (latest prices)
latest_prices = data.iloc[-1]

# Convert to DataFrame
prices_df = latest_prices.reset_index()
prices_df.columns = ["Ticker", "Price"]

# ✅ Filter out tickers with missing prices (NaN)
prices_df = prices_df.dropna().reset_index(drop=True)

# ✅ Create new list of valid tickers
new_tickers = prices_df["Ticker"].tolist()

print(prices_df.head())
print(f"\nValid tickers: {len(new_tickers)} / {len(tickers)}")
print("\nSample new_tickers:", new_tickers[:10])


/var/folders/92/sfk96cdd5y5754wnlxc583zc0000gn/T/ipykernel_32490/4181065963.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, period="1d")["Close"]
/opt/anaconda3/lib/python3.11/threading.py:910: RuntimeWarning: coroutine 'main' was never awaited
  self._invoke_excepthook = _make_invoke_excepthook()
[*********************100%***********************]  505 of 505 completed

85 Failed downloads:
['SNI', 'TWX', 'KORS', 'LUK', 'HES', 'EVHC', 'WYN', 'DPS', 'ANDV', 'WBA', 'NFX', 'SCG', 'FL', 'CBG', 'HCN', 'GGP', 'ANSS', 'COL', 'AET', 'BF.B', 'CSRA', 'ESRX', 'PCLN']: YFPricesMissingError('possibly delisted; no price data found  (period=1d)')
['ANTM', 'CBS', 'NLSN', 'GPS', 'ABC', 'CELG', 'ATVI', 'DISCK', 'CTXS', 'BRK.B', 'VIAB', 'ADS', 'COG', 'APC', 'LLL', 'BLL', 'XEC', 'FISV', 'DISCA', 'RHT', 'RE', 'PXD', 'MRO', 'DWDP', 'PDCO', 'CXO', 'HCP', 'ALXN', 'XL', 'AGN', 'MON', 'MYL', 'TSS', 'TIF', 'FBHS', 'BHGE', 'UTX', 'DISH', 'JNPR', 'A

  Ticker       Price
0      A  126.730003
1    AAL   12.950000
2    AAP   62.200001
3   AAPL  230.029999
4   ABBV  220.220001

Valid tickers: 420 / 505

Sample new_tickers: ['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM']


In [20]:
import pandas as pd
import numpy as np
import yfinance as yf

# ================== SETTINGS ==================
LOOKBACK_MONTHS = 12   # for SMA200 & 52w-high
AUTO_ADJUST = False    # be explicit to avoid surprises
# ==============================================

# ===== Helpers =====
def rsi(series, period: int = 14):
    delta = series.diff()
    up = np.where(delta > 0, delta, 0.0)
    down = np.where(delta < 0, -delta, 0.0)
    roll_up = pd.Series(up, index=series.index).ewm(alpha=1/period, adjust=False).mean()
    roll_down = pd.Series(down, index=series.index).ewm(alpha=1/period, adjust=False).mean()
    rs = roll_up / (roll_down.replace(0, np.nan))
    return 100 - (100 / (1 + rs))

def macd(series, fast=12, slow=26, signal=9):
    ema_fast = series.ewm(span=fast, adjust=False).mean()
    ema_slow = series.ewm(span=slow, adjust=False).mean()
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    hist = macd_line - signal_line
    return macd_line, signal_line, hist

def zscore(s, cap=3):
    z = (s - s.mean()) / (s.std(ddof=0) + 1e-9)
    return z.clip(-cap, cap) / cap  # scale to [-1,1]

# ===== Download OHLCV for all valid tickers (no batching) =====
data = yf.download(
    new_tickers,
    period=f"{LOOKBACK_MONTHS}mo",
    interval="1d",
    group_by="ticker",
    auto_adjust=AUTO_ADJUST,
    progress=False,
)

# --- Robust extraction of Close & High for any column shape ---
if isinstance(data.columns, pd.MultiIndex):
    lvl0 = data.columns.get_level_values(0)
    lvl1 = data.columns.get_level_values(1)
    if "Close" in lvl0 and "High" in lvl0:
        # Shape: (Field, Ticker)
        close = data["Close"]
        high  = data["High"]
    elif "Close" in lvl1 and "High" in lvl1:
        # Shape: (Ticker, Field)
        close = data.xs("Close", axis=1, level=1)
        high  = data.xs("High",  axis=1, level=1)
    else:
        raise RuntimeError("Could not find Close/High in downloaded data.")
else:
    # Single-ticker case → plain columns
    sym = new_tickers[0] if new_tickers else "TICKER"
    close = data[["Close"]].rename(columns={"Close": sym})
    high  = data[["High"]].rename(columns={"High": sym})

# Drop tickers with no data
close = close.dropna(axis=1, how="all")
high = high[close.columns]

if close.shape[1] == 0:
    raise RuntimeError("No usable tickers after download. Check new_tickers and connectivity.")

# ===== Build Features =====
prices = close.copy()
last = prices.tail(1).T.rename(columns={prices.index[-1]: "Price"})

sma50  = prices.rolling(50, min_periods=40).mean().tail(1).T.rename(columns={prices.index[-1]: "SMA50"})
sma200 = prices.rolling(200, min_periods=150).mean().tail(1).T.rename(columns={prices.index[-1]: "SMA200"})
rsi_14 = prices.apply(rsi, args=(14,)).tail(1).T.rename(columns={prices.index[-1]: "RSI14"})

macd_line, signal_line, hist = macd(prices)
macd_last   = macd_line.tail(1).T.rename(columns={prices.index[-1]: "MACD"})
signal_last = signal_line.tail(1).T.rename(columns={prices.index[-1]: "MACDSignal"})

ret_20 = prices.pct_change(20).tail(1).T.rename(columns={prices.index[-1]: "Ret20"})
ret_60 = prices.pct_change(60).tail(1).T.rename(columns={prices.index[-1]: "Ret60"})

# 52w-high proxy (use full window if <252 rows)
hi_lookback = prices.rolling(252, min_periods=min(60, prices.shape[0])).max()
hi_last = hi_lookback.tail(1).T.rename(columns={prices.index[-1]: "RollingHigh"})
dist_to_high = (last["Price"] / hi_last["RollingHigh"] - 1.0).to_frame(name="DistToHigh")

# Assemble feature table
feat = (
    last.join([sma50, sma200, rsi_14, macd_last, signal_last, ret_20, ret_60, dist_to_high])
        .replace([np.inf, -np.inf], np.nan)
        .dropna()  # ensure all features present
)

# ===== Scoring =====
W = {
    "ret20": 25,
    "ret60": 25,
    "dist_high": 15,
    "price_vs_sma50": 10,
    "sma50_vs_sma200": 10,
    "rsi": 15,
    "macd_signal": 10,
}

score = pd.Series(0.0, index=feat.index)
score += zscore(feat["Ret20"]) * W["ret20"]
score += zscore(feat["Ret60"]) * W["ret60"]
score += ((-feat["DistToHigh"]).clip(0, 1)) * W["dist_high"]
score += (feat["Price"] > feat["SMA50"]).astype(int) * W["price_vs_sma50"]
score += (feat["SMA50"] > feat["SMA200"]).astype(int) * W["sma50_vs_sma200"]
score += ((feat["RSI14"] - 50) / 50).clip(-1, 1) * W["rsi"]
score += (feat["MACD"] > feat["MACDSignal"]).astype(int) * W["macd_signal"]
score += (feat["MACD"] <= feat["MACDSignal"]).astype(int) * (-W["macd_signal"])

rank_df = feat.copy()
rank_df.insert(0, "Score", score.round(2))
rank_df = rank_df.sort_values("Score", ascending=False)

# ===== Output =====
top50 = rank_df.head(50)
bottom50 = rank_df.tail(50)

print("\n===== TOP 50 (Strongest) =====")
print(top50[["Score", "Price"]])

print("\n===== BOTTOM 50 (Weakest) =====")
print(bottom50[["Score", "Price"]])

rank_df.to_csv("sp500_ranked.csv")
print(f"\nSaved full ranking to sp500_ranked.csv with {len(rank_df)} tickers.")



===== TOP 50 (Strongest) =====
        Score        Price
Ticker                    
WDC     90.63    96.150002
STX     87.73   196.809998
STI     85.85     5.580000
ORCL    83.23   307.859985
KSS     77.38    16.139999
GLW     76.99    75.470001
M       75.24    17.459999
GOOGL   75.12   240.369995
GOOG    74.34   240.779999
AVGO    71.95   359.630005
NEM     71.40    79.650002
MU      70.61   150.570007
HPE     69.17    24.860001
DHI     65.94   181.009995
UNH     64.07   353.609985
NCLH    63.90    26.940001
VFC     59.70    15.480000
NTAP    59.44   125.089996
MHK     56.22   139.750000
FTI     55.97    40.299999
APTV    55.83    83.720001
APH     55.39   119.470001
VLO     55.39   157.589996
LRCX    52.87   115.580002
HP      52.23    20.900000
AMG     51.73   235.960007
LOW     51.69   272.399994
AZO     51.05  4354.540039
ABBV    50.64   220.220001
ALK     50.39    63.860001
HOG     50.04    30.700001
ZION    49.84    58.259998
GS      49.21   784.729980
MPC     49.16   182.309

In [ ]:
import alpaca_trade_api as tradeapi

# ================= SETTINGS =================
API_KEY = "PKYFT0K4HJKQTTHBPVJD"   
API_SECRET = "PWPgnoRzQGQMxdUtvzeRHwtMMahLztCU9IgeQifO"  
BASE_URL = "https://paper-api.alpaca.markets"

# Example tickers (replace with your own)
tickers = top50.index.tolist()
tickers1 = bottom50.index.tolist()

# =============================================

# Initialize API
api = tradeapi.REST(API_KEY, API_SECRET, BASE_URL, api_version='v2')

# BUY $1000 worth of each stock
for symbol in tickers:
    try:
        api.submit_order(
            symbol=symbol,
            notional=1000,      # $1000 worth of stock
            side='buy',
            type='market',
            time_in_force='day'
        )
        print(f"✅ Order to BUY $1000 of {symbol} placed")
    except Exception as e:
        print(f"❌ Error placing order for {symbol}: {e}")

# SHORT approx $1000 worth of each stock
for symbol in tickers1:
    try:
        # Get latest price
        quote = api.get_latest_trade(symbol)
        price = quote.price

        # Calculate whole share qty to short
        qty = int(1000 // price)   # floor division to ensure integer shares

        if qty > 0:
            api.submit_order(
                symbol=symbol,
                qty=qty,
                side='sell',       # short
                type='market',
                time_in_force='day'
            )
            print(f"✅ Order to SHORT {qty} shares of {symbol} (~$1000)")
        else:
            print(f"⚠️ Skipped {symbol} (price too high for $1000 short)")
    except Exception as e:
        print(f"❌ Error placing SHORT for {symbol}: {e}")
